In [111]:
import pandas as pd
import numpy as np

from scipy.stats import spearmanr, rankdata
from scipy.spatial.distance import pdist, squareform

import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split



In [114]:
def load_data(path):

    files = os.listdir(f'{path}\\img\\data\\')
    image = np.empty((len(files), 50, 50))
    sample = []
    id = []
    for i in range(len(files)):
        if files[i].split('.')[1] == 'txt':
            id.append(i)
            data = pd.read_csv(f'{path}\\img\\data\\{files[i]}', sep='\t', engine='c', na_values=['na', '-', ''], header=None, index_col=None)
            image[i, :, :] = data.values
            sample.append(files[i].split('.txt')[0])
    image = image[id, :, :]

    return image

In [151]:
def train_test_data(data_main, test_per = 0.2):
    rand_test_index= np.random.randint(0, data_main.shape[0], np.int64(data_main.shape[0]*test_per))
    test_data = data_main[rand_test_index]
    train_data = np.delete(data_main, rand_test_index, axis = 0)

    return train_data, test_data, rand_test_index


# train_data, test_data, test_indx = train_test_data(all_data, test_per=0.1)

In [152]:
# data_path = '..\\Results\\Table_To_Image_Conversion\\Test_1\\data'
data_path = "D:\\Research\\data\\processed_dataset" #lab

tissue_type = 'liver'
# tissue_type = 'pancreas'
# tissue_type = 'kidney'



all_data = load_data(path = f'{data_path}\\{tissue_type}')

In [216]:
y_target = pd.read_csv(f'{data_path}\\{tissue_type}\\miRNA_bulk.csv')
y_target = y_target.drop('Unnamed: 0',axis=1)

y_target = y_target.iloc[:,1:].drop(y_target.index[(y_target.iloc[:,1:].sum(axis = 1) < 90)]).T


y_target.shape

# y_target = y_target.iloc[:,1:].T
# y_target = y_target.values
# y_target

(981, 26)

In [217]:
X_train, X_test, y_train, y_test = train_test_split(all_data, y_target, test_size=0.2, random_state=1)

print(f'X_train: {X_train.shape}')
print(f'X_test: {X_test.shape}')
print(f'y_train: {y_train.shape}')
print(f'y_test: {y_test.shape}')




X_train: (784, 50, 50)
X_test: (197, 50, 50)
y_train: (784, 26)
y_test: (197, 26)


(784, 50, 50)

785

In [150]:
981*0.8

784.8000000000001